In [1]:
from pebble import ProcessPool
from concurrent.futures import TimeoutError

In [3]:
def process_all(func, arr, total=None, max_workers=None, timeout=None):
    with ProcessPool() as pool:
        future = pool.map(func, arr, timeout=timeout)

        iterator = future.result()
        results = []
        for i in progress_bar(range(len(arr)), total=len(arr)):
            try:
                result = next(iterator)
                results.append(result)
            except StopIteration:
                break  
            except TimeoutError as error:
                print("function took longer than seconds", error.args)
    return results

In [4]:
def some_task(n, *args, **kwargs):
    import time
    time.sleep(n)
    return n

In [5]:
import random
timings = [random.randint(0, 4) for i in range(100)]

In [6]:
from fastprogress.fastprogress import master_bar, progress_bar

In [7]:
process_all(some_task, timings, timeout=2)

function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than seconds ('Task timeout', 2)
function took longer than se

[2,
 1,
 2,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 2,
 1,
 2,
 1,
 1,
 0,
 1,
 2,
 1,
 2,
 2,
 0,
 1,
 1,
 2,
 1,
 2,
 1,
 0,
 2,
 1,
 2,
 0,
 0,
 2,
 1,
 0,
 2,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 1,
 2,
 1,
 0,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 2,
 2]

In [2]:
from pebble import ProcessPool
from concurrent.futures import TimeoutError

def fibonacci(n):
    if n == 0: return 0
    elif n == 1: return 1
    else: return fibonacci(n - 1) + fibonacci(n - 2)

with ProcessPool() as pool:
    future = pool.map(fibonacci, range(50), timeout=10)

    try:
        for n in future.result():
            print(n)
    except TimeoutError:
        print("TimeoutError: aborting remaining computations")
        future.cancel()

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418
317811
514229
832040
1346269
2178309
3524578
5702887
9227465
14930352
24157817
TimeoutError: aborting remaining computations


In [3]:
import os
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Process
from multiprocessing import Queue
from typing import Callable
from typing import Iterable
from typing import Dict
from typing import Any

In [4]:
class ProcessKillingExecutor:
    def __init__(self, max_workers: int=None):
        self.processes = max_workers or os.cpu_count()
        print('Num workers:', self.processes)

    def map(self,
            func: Callable,
            iterable: Iterable,
            timeout: float=None,
            callback_timeout: Callable=None,
            daemon: bool = True
            ) -> Iterable:
        executor = ThreadPoolExecutor(max_workers=self.processes)
        params = ({'func': func, 'fn_args': p_args, "p_kwargs": {},
                   'timeout': timeout, 'callback_timeout': callback_timeout,
                   'daemon': daemon} for p_args in iterable)
        return executor.map(self._submit_unpack_kwargs, params)

    def _submit_unpack_kwargs(self, params):
        """ unpack the kwargs and call submit """

        return self.submit(**params)

    def submit(self,
               func: Callable,
               fn_args: Any,
               p_kwargs: Dict,
               timeout: float,
               callback_timeout: Callable[[Any], Any],
               daemon: bool):
        p_args = fn_args if isinstance(fn_args, tuple) else (fn_args,)
        queue = Queue()
        p = Process(target=self._process_run,
                    args=(queue, func, fn_args,), kwargs=p_kwargs)

        if daemon:
            p.deamon = True

        p.start()
        p.join(timeout=timeout)
        if not queue.empty():
            return queue.get()
        if callback_timeout:
            callback_timeout(*p_args, **p_kwargs)
        if p.is_alive():
            p.terminate()
            p.join()

    @staticmethod
    def _process_run(queue: Queue, func: Callable[[Any], Any]=None,
                     *args, **kwargs):
        queue.put(func(*args, **kwargs))

In [5]:
def some_task(n, *args, **kwargs):
    import time
    time.sleep(n)
    return n ** 2

In [6]:
def fun_timeout(n):
    print('timeout:', n)

In [7]:
import random

In [8]:
timings = [random.randint(0, 4) for i in range(100)]

In [9]:
import concurrent

### Pebble Imp

In [ ]:
# executor = ProcessKillingExecutor(max_workers=None)
# generator = executor.map(some_task, timings, timeout=2,
#                          callback_timeout=fun_timeout)

# for elem in generator:
#     print(elem, 'sdfdsf')

https://stackoverflow.com/questions/20991968/asynchronous-multiprocessing-with-a-worker-pool-in-python-how-to-keep-going-aft

In [10]:

with ProcessPool() as pool:
    future = pool.map(some_task, timings, timeout=2)

    try:
        for n in future.result():
            print(n)
    except TimeoutError:
        print("TimeoutError:")
#         future.cancel()

1
0
4
TimeoutError:


In [13]:
with ProcessPool() as pool:
    future = pool.map(some_task, timings, timeout=2)
    
    iterator = future.result()
    results = []
    while True:
        try:
            result = next(iterator)
            results.append(result)
            print('Return result:', result)
        except StopIteration:
            break  
        except TimeoutError as error:
            print("function took longer than seconds", error.args[1])

Return result: 1
Return result: 0
Return result: 4
function took longer than seconds 2
function took longer than seconds 2
function took longer than seconds 2
Return result: 0
Return result: 4
Return result: 1
Return result: 1
function took longer than seconds 2
Return result: 4
function took longer than seconds 2
function took longer than seconds 2
function took longer than seconds 2
Return result: 4
Return result: 0
Return result: 1
Return result: 0
Return result: 1
Return result: 0
Return result: 1
Return result: 1
function took longer than seconds 2
Return result: 0
Return result: 1
function took longer than seconds 2
Return result: 1
function took longer than seconds 2
function took longer than seconds 2
Return result: 4
function took longer than seconds 2
function took longer than seconds 2
function took longer than seconds 2
Return result: 1
function took longer than seconds 2
Return result: 0
function took longer than seconds 2
function took longer than seconds 2
function took 

### PKE

In [15]:
executor = ProcessKillingExecutor(max_workers=None)
generator = executor.map(some_task, timings, timeout=2,
                         callback_timeout=fun_timeout)

for elem in generator:
    print(elem, 'sdfdsf')

Num workers: 96
timeout: 3
timeout: 2
None sdfdsf
0 sdfdsf
timeout: 4
timeout: 2
timeout: 4
timeout: 3
timeout: 2
timeout: 2
timeout: 3
timeout: 2
None sdfdsf
0 sdfdsf
None sdfdsf
0 sdfdsf
None sdfdsf
None sdfdsf
None sdfdsf
None sdfdsf
1 sdfdsf
timeout: 3
timeout: 3
None sdfdsf
0 sdfdsf
None sdfdsf
None sdfdsf
None sdfdsf
None sdfdsf
0 sdfdsf
timeout: 3
None sdfdsf
0 sdfdsf
0 sdfdsf
timeout: 3
timeout: 3
None sdfdsf
timeout: 4
None sdfdsf
None sdfdsf
1 sdfdsf
0 sdfdsf
0 sdfdsf
1 sdfdsf
timeout: 3
None sdfdsf
1 sdfdsf
1 sdfdsf
0 sdfdsf
0 sdfdsf
1 sdfdsf
0 sdfdsf
timeout: 3
timeout: 3
None sdfdsf
1 sdfdsf
None sdfdsf
1 sdfdsf
1 sdfdsf
1 sdfdsf
1 sdfdsf
timeout: 3
None sdfdsf
0 sdfdsf
1 sdfdsf
timeout: 3
None sdfdsf
timeout: 2
None sdfdsf
timeout: 4
None sdfdsf
0 sdfdsf
timeout: 2
timeout: 2
None sdfdsf
0 sdfdsf
timeout: 3
None sdfdsf
1 sdfdsf
None sdfdsf
0 sdfdsf
0 sdfdsf
timeout: 2
None sdfdsf
1 sdfdsf
timeout: 2
timeout: 4
1 sdfdsf
timeout: 3
None sdfdsf
1 sdfdsf
None sdfdsf
0 sdfdsf


In [14]:
executor = ProcessKillingExecutor(max_workers=None)
generator = [executor.submit(some_task, i, p_kwargs={}, timeout=4,
                         callback_timeout=fun_timeout, daemon=True) for i in timings]
for elem in concurrent.futures.as_completed(generator):
    print(elem, 'sdfdsf')

Num workers: 96
timeout: 4
timeout: 4


Process Process-117:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-12-1644fa1df89c>", line 52, in _process_run
    queue.put(func(*args, **kwargs))
  File "<ipython-input-3-a6a9aafe4eb5>", line 3, in some_task
    time.sleep(n)


KeyboardInterrupt: 

KeyboardInterrupt


In [ ]:

def process_all_std(func, arr, total=None, max_workers=None, timeout=None):
    "Process array in parallel"
    if total is None: total = len(arr)
    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(func,o) for i,o in enumerate(arr)]
        for f in progress_bar(concurrent.futures.as_completed(futures, timeout=timeout), total=total):
            res = f.result(timeout=timeout)
            if res is not None:
                results.append(res)
    return results

In [8]:
# for elem in generator:
#     print(elem)

timeout: 10
9
None

4
0
timeout: 6
None
timeout: 6
None
timeout: 6
None
timeout: 9
None
timeout: 7
None
None
timeout: 4
None
1
0
timeout: 9
None
timeout: 10
None
None
timeout: 9
None
1
timeout: 6
None
timeout: 6
None
timeout: 10
None
timeout: 9
None
None
timeout: 6
None
1
None
1
timeout: 9
None
0
9

0
9
timeout: 7
None
4
timeout: 7
None
timeout: 7
None

4
4
timeout: 6
None
0
timeout: 10
None
timeout: 9
None
timeout: 4
None

4

9
4
4

4

timeout: 10
None
0

0
timeout: 10
None
0
None
0
timeout: 4
None
1
timeout: 5
None
timeout: 4
None
timeout: 7
None
timeout: 10
None

timeout: 4
None
4
timeout: 8
None
timeout: 5
None
timeout: 9
None
timeout: 10
None

1
timeout: 10
None
timeout: 6
None

timeout: 4
None
1
0
0
timeout: 8
None
timeout: 8
None
timeout: 5
None
timeout: 10
None

9
timeout: 4
None
timeout: 7
None

None
4
0
timeout: 10
None
timeout: 7
None

1
timeout: 5
None
timeout: 9
None
timeout: 8
None

timeout: 10
None
0
timeout: 7
timeout: 6
timeout: 6
None
None
0
timeout: 6
None
timeout: 6

None
timeout: 5
None
timeout: 6
None
timeout: 9
None
timeout: 9
None
timeout: 6
None
timeout: 8
None

1
timeout: 4
None
timeout: 10
None
None
4
timeout: 4
None
timeout: 10
None
None
timeout: 6
None
0

0
timeout: 7
None
0
timeout: 4
None
None
timeout: 5
None
0
None
0
timeout: 10
None
1
timeout: 9
None
timeout: 5
None
timeout: 8
None
None
9
timeout: 6
None
timeout: 10
None
None
9

timeout: 4
None
0
4
4
timeout: 8
None
9
timeout: 4
None
None
timeout: 5
None
0
4
None
0
9
9
timeout: 9
None
4
timeout: 5
None
timeout: 9
None
None
timeout: 7
None
9
None
9
4
0
0
4
0
0
9
1
4
timeout: 5
4
None
9
timeout: 4
None
timeout: 7
None
timeout: 10
None
timeout: 9
None
None
None
9
0
1
timeout: 7
None
None
timeout: 9
None
1
timeout: 9
None
None
timeout: 8
None
0
0
None
0
0
timeout: 4
None
0
4
1
0
timeout: 8
None
timeout: 7
None


1
0
timeout: 8
None
timeout: 10
None
None
4
timeout: 6
None
timeout: 7
None
None
timeout: 7
None
4
1
timeout: 4
None
timeout: 7
None

9
timeout: 10
None
None
timeout: 10
None
4
1
t

None

timeout: 9
None
4
4
timeout: 10
None
None
timeout: 5
None
1
timeout: 8
None
timeout: 6
None
timeout: 8
None
timeout: 7
None
timeout: 4
None
timeout: 5
None
None
1
None
9
timeout: 4
None
timeout: 4
None
timeout: 6
None
None
timeout: 10
None
0
None
0
timeout: 7
None
0
4
timeout: 6
None
timeout: 9
None
timeout: 6
None
timeout: 5
None
timeout: 6
None
timeout: 4
None
None
1
timeout: 9
None
timeout: 8
None
None
timeout: 7
None
9
4
timeout: 6
None
timeout: 4
None
timeout: 6
None
timeout: 9
None
timeout: 4
None
timeout: 9
None

4
4
timeout: 4
None
0
timeout: 9
None
None

0
9
1
timeout: 5

1
None
9
9
None
1
0
timeout: 8
None
timeout: 8
None
None
timeout: 8
None
4
None
4
9
4
timeout: 8
None
timeout: 8
None
None
1
timeout: 6
None
timeout: 6
None
timeout: 10
None
timeout: 9
None
None
timeout: 10
None
4
timeout: 6
None
timeout: 4
None
timeout: 10
None
timeout: 6
None
timeout: 7
None
None
timeout: 4
None
1
timeout: 8
None
timeout: 5
None
None
timeout: 10
None
1
timeout: 6
None
timeout: 7
None


None
9
None
9
timeout: 9
None
0
timeout: 10
None


0
9
1
timeout: 9
None
timeout: 5
None

9
timeout: 5
None
timeout: 4
None
timeout: 9
None
timeout: 9
None
None
None
9
0
timeout: 5
None
timeout: 4
None
timeout: 5
None
timeout: 5
None
timeout: 10
None
timeout: 6
None
timeout: 9
None
timeout: 10
None

timeout: 7
None
1
timeout: 5
None
timeout: 8
None
timeout: 7
None

9
timeout: 8
None
timeout: 6
None
timeout: 5
None
timeout: 9
None


9
4
timeout: 8
None
timeout: 6
None
None
4
timeout: 8
None
timeout: 6
None
timeout: 4
None

timeout: 8
None
1
timeout: 7
None
timeout: 7
None

1

9
timeout: 4
None
timeout: 8
None
timeout: 5
None
timeout: 10
None
None
timeout: 8
None
0

0
timeout: 9
None
1
timeout: 6
None

timeout: 9
None
0

0
timeout: 6
None
4

4
4
1
0
4
1
0
1
0
1
0
9
4
timeout: 9
None
timeout: 8
None
timeout: 10
None
timeout: 4
None
None
4
4

4
0

0
timeout: 9
None
0
0
timeout: 9
None
timeout: 9
None
timeout: 6
None
None
9
timeout: 9
None
timeout: 6
None

9
None
9
timeout: 4
None
timeout: 

None
timeout: 4
None
timeout: 7
None
timeout: 9
None

1
timeout: 4
None
timeout: 5
None
timeout: 8
None
timeout: 4
None
timeout: 7
None
timeout: 8
None

1

timeout: 8
None
0
1
0
timeout: 5
None
timeout: 5
None
timeout: 7
None
timeout: 8
None
timeout: 9
None
timeout: 6
None
None

9
0
4
timeout: 5
None
timeout: 10
None

timeout: 6
None
4
timeout: 10
None
timeout: 10
None
None
timeout: 9
None
0
timeout: 4
None
timeout: 6
None

timeout: 9
None
0
timeout: 10
None
timeout: 10
None
timeout: 6
None
timeout: 10
None

4
timeout: 10
None
timeout: 9
None
timeout: 4
None

timeout: 8
None
0
timeout: 9
None
timeout: 10
None

9
timeout: 8
None
timeout: 4
None

9
timeout: 9
None
timeout: 8
None
timeout: 9
None
timeout: 8
None

4
timeout: 10
None
timeout: 9
None

1
1
timeout: 4
None
9
None
9
timeout: 9
None
1
1
0
timeout: 8
None
timeout: 6
None
timeout: 4
None

9
timeout: 4
None
timeout: 10
None
None
4
timeout: 8
None
timeout: 9
None
timeout: 7
None
timeout: 10
None
timeout: 7
None

timeout: 4
None
1

1

...

None
9
9
None
9
9
timeout: 7
None
timeout: 6
None
None
timeout: 8
None
1
0
0
4
4
timeout: 8
None
9
timeout: 7
None
timeout: 5
None
None
timeout: 5
None
4
timeout: 8
None
timeout: 8
None
timeout: 5
None
None
9
timeout: 9
None
timeout: 5
None
timeout: 8
None
timeout: 6
None
timeout: 6
None
timeout: 10
None
None
4
timeout: 8
None
4
None
9
1
timeout: 7
None
timeout: 6
None
timeout: 9
None
timeout: 6
None
None
timeout: 10
None
4
None
4
timeout: 8
None
4
1
timeout: 6
None
timeout: 4
None
None
None
9
1

1
timeout: 8
None
0
timeout: 9
None
None
timeout: 4
None
1
timeout: 9
None
None
timeout: 5
None
1
1
1
timeout: 7
None
9
0
4
timeout: 5
None
timeout: 4
None
None
4None

1


4
0

0
timeout: 4
None
4
timeout: 7
None
timeout: 4
None
timeout: 4
None
timeout: 6
None
timeout: 5
None
timeout: 5
None
None
9
timeout: 5
None
timeout: 8
None
timeout: 10
None
None
timeout: 10
None
0
timeout: 6
None
None
timeout: 10
None
4
None
4
timeout: 8
None
1
timeout: 7
None
timeout: 6
None
None
9
9
timeout: 4
None
0
t

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



timeout: 6
None
timeout: 4
None

9
None
9
9
4

4
4
1
4
9
timeout: 10
None

timeout: 5
None
1
timeout: 4
None
timeout: 5
None
timeout: 4
None

timeout: 9
None
0
9
timeout: 7
None
timeout: 7
None

timeout: 10
None
1
timeout: 5
None

timeout: 8
None
1
None
1
timeout: 5
None
1
timeout: 8
None
timeout: 7
None

timeout: 9
None
0
timeout: 5
None
timeout: 4
None
timeout: 8
None

None
9
0
9
0
timeout: 7
None
9
9
timeout: 4
None
9
9
timeout: 9
None
timeout: 10
None

9
9
timeout: 7
None
1

1
timeout: 6
None
0
timeout: 5
None
timeout: 6
None
timeout: 4
None
timeout: 5
None
timeout: 9
None

timeout: 5
None
1
timeout: 4
None
timeout: 5
None
timeout: 5
None

9
9
timeout: 8
None
0
0
timeout: 4
None
timeout: 4
None
timeout: 6
None
timeout: 6
None
timeout: 9
None

4
timeout: 5
None
timeout: 6
NoneNone

4
timeout: 6
None
timeout: 6
None
timeout: 6
None
timeout: 10
None
timeout: 10
None
timeout: 8
None
timeout: 6
None
timeout: 8
None
None
timeout: 9
None
0
9
timeout: 4
None
timeout: 7
None
None
9
timeout:

None
timeout: 6
None
timeout: 6
None

4

9
timeout: 10
None
timeout: 6
None
timeout: 5
None
timeout: 4
None
timeout: 6
None

4

timeout: 5
None
0
4
timeout: 10
None
timeout: 8
None

timeout: 8
None
0
timeout: 6
None
timeout: 4
None
timeout: 5
None
timeout: 7
None
timeout: 6
None

timeout: 8
None
0
timeout: 6
None

timeout: 10
None
0
timeout: 10
None

timeout: 9
None
1
None
1
timeout: 10
None
0

0
timeout: 10
None
1
timeout: 6
None
timeout: 6
None
timeout: 10
None
timeout: 4
None
timeout: 6
None
timeout: 5
None

timeout: 6
None
4
timeout: 9
None
timeout: 9
None
timeout: 5
None

None
9
1
1
timeout: 6
None
4

4
timeout: 5
None
0
4
timeout: 5
None
timeout: 4
None

timeout: 9
None
4
4
4
timeout: 6
None
1
None
1
timeout: 8
None
0
1
1
timeout: 10
None
timeout: 8
None

1
1
timeout: 7
None
0
timeout: 8
None

0
9
timeout: 9
None

timeout: 5
None
4
timeout: 6
None
timeout: 5
None
timeout: 10
None
timeout: 6
None

timeout: 9
None
4
timeout: 4
None
timeout: 9
None

4

timeout: 8
None
4
0
timeout: 6

9
timeout: 8
None
4
timeout: 4
None
timeout: 8
None

4
timeout: 8
None
timeout: 9
None
timeout: 8
None
timeout: 8
None
timeout: 10
None

timeout: 9
None
0
timeout: 9
None
timeout: 8
None

4

timeout: 6
None
0
4
timeout: 8
None
timeout: 5
None
timeout: 7
None
timeout: 4
None
None
timeout: 9
None
1
timeout: 8
None
timeout: 9
None
timeout: 9
None
timeout: 4
None


9
1
9
1
timeout: 6
None
4
timeout: 4
None

timeout: 9
None
0
timeout: 5
None
timeout: 9
None
None
timeout: 5
None
4
9
9
timeout: 5
None
1
timeout: 6
None
timeout: 8
None
timeout: 10
None
timeout: 6
None
timeout: 6
None
timeout: 5
NoneNone

1
timeout: 8
None
timeout: 8
None

1

timeout: 5
None
0
9
4
